### Make cells wider

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Imports

In [2]:
import os

In [3]:
os.chdir('../..') # local

In [4]:
os.environ['NUMEXPR_MAX_THREADS'] = '32'
os.environ['NUMEXPR_NUM_THREADS'] = '16'

In [5]:
%%time
from dev.scripts.ML_utils import * # run if on local machine
from dev.scripts.numerai_utils import *

from dev.scripts.trading_utils import * # run if on local machine

CPU times: user 1.65 s, sys: 1.97 s, total: 3.62 s
Wall time: 1.37 s


In [6]:
import configparser
config = configparser.ConfigParser()
config.read('numerai/numerai_keys.ini')

OUTPUT_PATH = '/media/melgazar9/HDD_10TB/trading/data/numerai/'

### Load in the df created in build_numerai_dataset.ipynb
train / val columns are already populated into the dataset, but later I will create a test set and use the validation to improve hyperparameters 

In [7]:
%%time
DF_NUMERAI_PATH = '/media/melgazar9/HDD_10TB/trading/data/yfinance/df_numerai_2021-04-11.feather'
df_numerai = pd.read_feather(DF_NUMERAI_PATH)

df_numerai.tail(2)

CPU times: user 4.27 s, sys: 12.2 s, total: 16.5 s
Wall time: 1.08 s


,date,yahoo_ticker,adj_close_1d,close_1d,high_1d,low_1d,open_1d,volume_1d,adj_close_1h_0,adj_close_1h_1,...,prev1_target_rolling_mean,open_1d_rolling_mean,prev1_target_HL5_rolling_mean,high_1d_ewm_mean,low_1d_ewm_mean,adj_close_1d_ewm_mean,volume_1d_ewm_mean,prev1_target_ewm_mean,open_1d_ewm_mean,prev1_target_HL5_ewm_mean
2633677,2021-03-26,ZZZ.TO,32.060001,32.060001,32.060001,31.625,31.709999,40900.0,NaN,NaN,...,0.516667,27.616333,2.000000,31.963528,31.342508,31.912844,73758.914766,0.525030,31.537437,2.780531
2633678,2021-03-26,ZZZ.TO,32.060001,32.060001,32.060001,31.625,31.709999,40900.0,NaN,NaN,...,0.516667,27.844000,2.033333,32.027844,31.530836,32.010949,51852.971589,0.508343,31.652478,2.260177


### Select input features
OHLC are not valid input features since they are not stationary and there is no upper bound. I will only be selecting stationary or categorical features as input features.

In [8]:
import re

In [9]:
non_features = list(filter(re.compile("open_.*[0-9]$|.*close_.*[0-9]$|low_.*[0-9]$|.*high_.*[0-9]$").match, df_numerai.columns.tolist()))
non_features[0:10]

['adj_close_1h_0',
 'adj_close_1h_1',
 'adj_close_1h_2',
 'adj_close_1h_3',
 'adj_close_1h_4',
 'adj_close_1h_5',
 'adj_close_1h_6',
 'adj_close_1h_7',
 'adj_close_1h_8',
 'adj_close_1h_9']

In [83]:
df_numerai['adj_close_1d_ewm_mean']

0          2475.666504
1          2800.644836
2          2896.690993
3          3240.220953
4          3072.756077
              ...     
2633674      30.726770
2633675      31.395590
2633676      31.618530
2633677      31.912844
2633678      32.010949
Name: adj_close_1d_ewm_mean, Length: 2633679, dtype: float64

In [9]:
df_numerai.columns.tolist()

['date',
 'yahoo_ticker',
 'adj_close_1d',
 'close_1d',
 'high_1d',
 'low_1d',
 'open_1d',
 'volume_1d',
 'adj_close_1h_0',
 'adj_close_1h_1',
 'adj_close_1h_2',
 'adj_close_1h_3',
 'adj_close_1h_4',
 'adj_close_1h_5',
 'adj_close_1h_6',
 'adj_close_1h_7',
 'adj_close_1h_8',
 'adj_close_1h_9',
 'adj_close_1h_10',
 'adj_close_1h_11',
 'adj_close_1h_12',
 'adj_close_1h_13',
 'adj_close_1h_14',
 'adj_close_1h_15',
 'adj_close_1h_16',
 'adj_close_1h_17',
 'adj_close_1h_18',
 'adj_close_1h_19',
 'adj_close_1h_20',
 'adj_close_1h_21',
 'adj_close_1h_22',
 'adj_close_1h_23',
 'close_1h_0',
 'close_1h_1',
 'close_1h_2',
 'close_1h_3',
 'close_1h_4',
 'close_1h_5',
 'close_1h_6',
 'close_1h_7',
 'close_1h_8',
 'close_1h_9',
 'close_1h_10',
 'close_1h_11',
 'close_1h_12',
 'close_1h_13',
 'close_1h_14',
 'close_1h_15',
 'close_1h_16',
 'close_1h_17',
 'close_1h_18',
 'close_1h_19',
 'close_1h_20',
 'close_1h_21',
 'close_1h_22',
 'close_1h_23',
 'high_1h_0',
 'high_1h_1',
 'high_1h_2',
 'high_1h

### Preprocess features

In [ ]:
PRESERVE_VARS = ['date', 'yahoo_ticker', '']

In [ ]:
feature_transformer = PreprocessFeatures(preserve_vars = PRESERVE_VARS, **FE_PARAMS).fit(X_train, y_train)